In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
import math
import sys


options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)


# Disable images
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver.get('https://www.sharesansar.com/floorsheet')

# company = "NABIL"  
# Select company from dropdown
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="select2-company-container"]'))).click()
try:
    driver.find_element(By.CLASS_NAME, 'select2-search__field').send_keys("Nabil Bank Limited / NABIL")
except NoSuchWindowException:
    pass
driver.find_element(By.ID, 'btn_flsheet_submit').click()
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'myTable_processing')))
WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, 'myTable_processing')))


# Select '500' from dropdown
Select(driver.find_element(By.NAME, 'myTable_length')).select_by_visible_text('500')
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'myTable_processing')))
WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, 'myTable_processing')))

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', id='myTable')
a = soup.find('div', class_='dataTables_info').text
total_page = math.ceil(int(a.split(' ')[5].replace(',', '')) / 500)

company_data = []

for index in range(1, total_page + 1):
    if index > 1:
        driver.find_element(By.LINK_TEXT, 'Next').click()
        WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'myTable_processing')))
        WebDriverWait(driver, 10).until(EC.invisibility_of_element_located((By.ID, 'myTable_processing')))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table', id='myTable')

    output_rows = []
    if len(table.findAll('tr')) != 4:
        for table_row in table.findAll('tr'):
            columns = table_row.findAll('td')
            output_row = [column.text for column in columns]
            output_rows.append(output_row)
        
        headers_list = [header.text.replace('\n', '') for header in table.find_all('th')]

        todays_price_dataframe = pd.DataFrame(output_rows)[1:]
        todays_price_dataframe.columns = headers_list
        todays_price_dataframe.replace('\n', "", regex=True, inplace=True)
        todays_price_dataframe['Transaction No.'] = todays_price_dataframe['Transaction No.'].astype(str) + "\t"
        todays_price_dataframe = todays_price_dataframe[:-2]
        company_data.append(todays_price_dataframe)

df = pd.concat(company_data, ignore_index = True)
print(f'The program was run at {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

The program was run at 2025-01-29 00:05:31
